In [1]:
import json
import argparse
import numpy as np
import pandas as pd
import seaborn as sns
from datasets import load_dataset, Dataset, load_from_disk
import matplotlib.pyplot as plt 
from transformers import AutoTokenizer

/home/yilong/miniconda3/envs/vllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# KK

In [2]:
train_test_ds3 = load_dataset('parquet', data_files='../data/kk/instruct/3ppl/train.parquet')['train']
train_test_ds4 = load_dataset('parquet', data_files='../data/kk/instruct/4ppl/train.parquet')['train']

In [3]:
dfkk3 = pd.DataFrame(train_test_ds3)
dfkk3.head(2)

,quiz,names,knight_knave,solution,solution_text,solution_text_format,cot_head,cot_repeat_steps,cot_foot,statements,index,data_source,prompt,ability,reward_model,extra_info
0,A very special island is inhabited only by kni...,"[Michael, Zoey, Ethan]","{'Knave': 'Knave', 'Knight': 'Knight', 'a_knav...","[True, True, True]","Michael is a knight, Zoey is a knight, and Eth...",(1) Michael is a knight\n(2) Zoey is a knight\...,"Let's think step by step, by considering wheth...",[Assume Michael is a knight. No contradiction ...,This leads to a feasible solution.,"(('<=>', ('telling-truth', 2), ('telling-truth...",100,kk_logic,[{'content': '<|im_start|>system You are a hel...,logic,{'ground_truth': {'solution_text_format': '(1)...,"{'index': 0, 'split': 'train'}"
1,A very special island is inhabited only by kni...,"[Michael, Mason, Jacob]","{'Knave': 'Knave', 'Knight': 'Knight', 'a_knav...","[True, False, True]","Michael is a knight, Mason is a knave, and Jac...",(1) Michael is a knight\n(2) Mason is a knave\...,"Let's think step by step, by considering wheth...",[Assume Michael is a knight. No contradiction ...,This leads to a feasible solution.,"(('not', ('telling-truth', 1)), ('and', ('tell...",101,kk_logic,[{'content': '<|im_start|>system You are a hel...,logic,{'ground_truth': {'solution_text_format': '(1)...,"{'index': 1, 'split': 'train'}"


In [4]:
dfkk4 = pd.DataFrame(train_test_ds4)
dfkk4.head(2)

,quiz,names,knight_knave,solution,solution_text,solution_text_format,cot_head,cot_repeat_steps,cot_foot,statements,index,data_source,prompt,ability,reward_model,extra_info
0,A very special island is inhabited only by kni...,"[Ethan, Abigail, David, Noah]","{'Knave': 'Knave', 'Knight': 'Knight', 'a_knav...","[False, True, True, True]","Ethan is a knave, Abigail is a knight, David i...",(1) Ethan is a knave\n(2) Abigail is a knight\...,"Let's think step by step, by considering wheth...",[Assume Ethan is a knight. No contradiction is...,This leads to a feasible solution.,"(('or', ('lying', 2), ('lying', 1)), ('->', ('...",100,kk_logic,[{'content': '<|im_start|>system You are a hel...,logic,{'ground_truth': {'solution_text_format': '(1)...,"{'index': 0, 'split': 'train'}"
1,A very special island is inhabited only by kni...,"[Elizabeth, Sophia, Mason, Abigail]","{'Knave': 'Knave', 'Knight': 'Knight', 'a_knav...","[True, False, False, False]","Elizabeth is a knight, Sophia is a knave, Maso...",(1) Elizabeth is a knight\n(2) Sophia is a kna...,"Let's think step by step, by considering wheth...",[Assume Elizabeth is a knight. No contradictio...,This leads to a feasible solution.,"(('and', ('lying', 3), ('lying', 2)), ('->', (...",101,kk_logic,[{'content': '<|im_start|>system You are a hel...,logic,{'ground_truth': {'solution_text_format': '(1)...,"{'index': 1, 'split': 'train'}"


In [5]:
print('len 3ppl:{}\nlen 4ppl:{}'.format(len(dfkk3), len(dfkk4)))

len 3ppl:900
len 4ppl:900


In [6]:
dfkk = pd.concat([dfkk3, dfkk4], ignore_index=True)

In [7]:
print(dfkk.prompt[0][0]['content'])

<|im_start|>system
You are a helpful assistant. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and<answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>.  Now the user asks you to solve a logical reasoning problem. After thinking, when you finally reach a conclusion, clearly state the identity of each character within <answer> </answer> tags. i.e., <answer> (1) Zoey is a knight
(2) ... </answer>.
<|im_end|>
<|im_start|>user
A very special island is inhabited only by knights and knaves. Knights always tell the truth, and knaves always lie. You meet 3 inhabitants: Michael, Zoey, and Ethan. Michael was heard saying, "Ethan is a knight if and only if Michael is a knight". "Zoey is a knight or Ethan is a knight," Zoey mentioned. Ethan asserted: "Michael is a knave if and only if Zoey is a knave". So 

In [8]:
SYSTEM_PROMPT = """You are a helpful assistant. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>."""

In [9]:
dfk = dfkk.loc[:, ['ability']].copy()
dfk['story']=dfkk.quiz
dfk['answer']=dfkk.solution_text_format

In [10]:
def generate_prompt(x, i):
    data = { # type: ignore
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': x['story']}
            ],
            'answer': x['answer']
        }
    return data
prompt = []


for i, row in dfk.iterrows():
    prompt.append([
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': row['story']+'Clearly state the identity of each character in answer. i.e., (1) character_name is a character_identity (2) ... '}
            ])

dfk['prompt']=prompt

In [11]:
dfk.head(2)

,ability,story,answer,prompt
0,logic,A very special island is inhabited only by kni...,(1) Michael is a knight\n(2) Zoey is a knight\...,"[{'role': 'system', 'content': 'You are a help..."
1,logic,A very special island is inhabited only by kni...,(1) Michael is a knight\n(2) Mason is a knave\...,"[{'role': 'system', 'content': 'You are a help..."


In [12]:
len(dfk)

1800

In [13]:
np.max(list(map(lambda x: len(x.split()), dfk.story)))

99

In [14]:
print(dfk.prompt[0][1]['content'])
print(dfk.answer[0])

A very special island is inhabited only by knights and knaves. Knights always tell the truth, and knaves always lie. You meet 3 inhabitants: Michael, Zoey, and Ethan. Michael was heard saying, "Ethan is a knight if and only if Michael is a knight". "Zoey is a knight or Ethan is a knight," Zoey mentioned. Ethan asserted: "Michael is a knave if and only if Zoey is a knave". So who is a knight and who is a knave?Clearly state the identity of each character in answer. i.e., (1) character_name is a character_identity (2) ... 
(1) Michael is a knight
(2) Zoey is a knight
(3) Ethan is a knight


# ToM

In [15]:
train_nt = load_dataset('parquet', data_files='cleaned_train_nt_with_structure_384.parquet')['train']
train_ft = load_dataset('parquet', data_files='cleaned_train_ft_with_structure_384.parquet')['train']

In [16]:
train_nt['prompt'][0]

[{'content': 'You are a helpful assistant. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>.',
  'role': 'system'},
 {'content': 'Story: The tourist information center, a cozy hub of local knowledge and traveler activity, stood ready to serve as a central gathering point. Its warm wooden accents and large windows provided a welcoming atmosphere amidst the bustling downtown area. Two familiar faces, Abigail and Andrew, converged on the tourist information center, bringing with them their unique expertise and infectious enthusiasm for the local scene. Abigail\'s voice carried through the center, weaving a vivid tapestry of local treasures, including the iconic lighthouse and the quaint seaside promenade. As she spoke, her eyes sparkled with 

In [17]:
train_ft['prompt'][0]

[{'content': 'You are a helpful assistant. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think><answer> answer here </answer>.',
  'role': 'system'},
 {'content': "Story: Lauren entered the back room of the coffee shop. Anthony entered the back room of the coffee shop. Lauren moved the bag of gourmet coffee beans to the wooden crate, which is also located in the back room of the coffee shop. Anthony moved the bag of gourmet coffee beans to the canvas tote bag, which is also located in the back room of the coffee shop. Lauren placed a label that says 'decaf' on the bag of gourmet coffee beans Lauren moved the bag of gourmet coffee beans to the cardboard box, which is also located in the back room of the coffee shop.\n Question:In which container is the bag of gourmet coffee

In [18]:
dfnt = pd.DataFrame(train_nt)
dfft = pd.DataFrame(train_ft)

In [19]:
len(dfft)

4429

In [20]:
len(dfnt)

5722

In [21]:
print(len(dfnt), len(dfnt.loc[dfnt.answer.isin(['yes'])]), len(dfnt.loc[dfnt.answer.isin(['knows about it'])]), len(dfnt.loc[dfnt.answer.isin(['does not know about it'])]))
print(dfnt.loc[dfnt.answer.isin(['knows about it'])]['question'].values[0])
print(dfnt.loc[dfnt.answer.isin(['yes'])]['question'].values[0])
print(dfnt.loc[~dfnt.answer.isin(['knows about it', 'yes'])]['question'].values[0])

5722 1584 791 0
What does Abigail think about Alexis's belief on festival venue layout? (knows about it / does not know about it)
Does Abigail think that Jasmine believes that the silver letter opener has a microscopic identification code etched at the base of the handle? Answer yes or no.
In which room does Abigail think that Alexis will search for the leather briefcase?


In [22]:
NT = 300
NT_yes = 20
NT_know = 20
FT = 1500

In [23]:
dfn = pd.concat([
    dfnt.loc[dfnt.answer.isin(['yes'])].sample(n = NT_yes, replace=False, ignore_index=True, random_state=42),
    dfnt.loc[dfnt.answer.isin(['knows about it'])].sample(n = NT_know, replace=False, ignore_index=True, random_state=42),
    dfnt.loc[~dfnt.answer.isin(['knows about it', 'yes'])].sample(n = NT-NT_know-NT_yes, replace=False, ignore_index=True, random_state=42),
    ],
    ignore_index=True)
dff = dfft.sample(n = FT, replace=False, ignore_index=True, random_state=42)

In [24]:
len(dfn)

300

In [25]:
df = pd.concat([dfk, dfn.loc[:, ['ability', 'story', 'answer', 'prompt']], dff.loc[:, ['ability', 'story', 'answer', 'prompt']] ])
df = df.sample(frac=1, replace=False, ignore_index=True)

In [89]:
#df.to_csv('ToM_KK_train.csv', index=False)

In [90]:
data =Dataset.from_pandas(df)

In [91]:
data

Dataset({
    features: ['ability', 'story', 'answer', 'prompt'],
    num_rows: 3800
})

In [63]:
data.to_parquet('ToM_KK_train.parquet')

Creating parquet from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 416.31ba/s]


4972263

In [26]:
dft = pd.concat([dfn.loc[:, ['ability', 'story', 'answer', 'prompt']], dff.loc[:, ['ability', 'story', 'answer', 'prompt']] ])
dft = dft.sample(frac=1, replace=False, ignore_index=True)

In [28]:
len(np.unique(dft.story))

495

In [29]:
data_tom = load_dataset('parquet', data_files='ToM_2k.parquet')['train']

Generating train split: 2000 examples [00:00, 184645.02 examples/s]


In [31]:
df_tom = pd.DataFrame(data_tom)

In [43]:
ans = (df_tom.groupby('answer').story.count()/len(df_tom)).reset_index()

In [48]:
ans.loc[ans.story>0.02]

,answer,story
39,does not know about it,0.2925
52,knows about it,0.1725
73,no,0.0635
81,plastic storage bin,0.0225
101,yes,0.1630


In [49]:
ans.loc[ans.story>0.02].story.sum()

0.714

In [ ]:
ans.loc[ans.story>50]